In [ ]:
from confluent_kafka import Producer, Consumer, KafkaError

: 

In [ ]:
# Kafka configuration
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'python-example-group',
    'auto.offset.reset': 'earliest'
}

# Producer setup
producer = Producer({'bootstrap.servers': 'localhost:9092'})

In [ ]:
# Util methods
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')

In [ ]:
# Produce messages to topic 'test-topic'
for i in range(10):
    producer.produce('test-topic', key=str(i), value=f'Message #{i}', callback=delivery_report)
    producer.poll(0)  # Serve delivery callback

producer.flush()

In [ ]:
# Consumer setup
consumer = Consumer(conf)
consumer.subscribe(['test-topic'])

print('Consuming messages from topic "test-topic"...')

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                print(f'Reached end of partition {msg.partition()}')
            else:
                print(f'Error while consuming message: {msg.error()}')
        else:
            print(f'Received message: Key={msg.key().decode("utf-8")}, Value={msg.value().decode("utf-8")}')
except KeyboardInterrupt:
    pass
finally:
    consumer.close()